## Import Libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

! pip install BeautifulSoup4
from bs4 import BeautifulSoup

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.0

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.0

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Libraries imported.


## Scraping Data from Wiki

In [2]:
import pandas as pd
import wikipedia as wp
 
#Get the html source
html = wp.page("Asian population of London").html().encode("UTF-8")
London_df = pd.read_html(html)[4]
London_df.head()

,Rank,London Borough,Indian Population,Pakistani Population,Bangladeshi Population,Chinese Population,Other Asian Population,Total Asian Population
0,1,Newham,42484,30307,37262,3930,19912,133895
1,2,Redbridge,45660,31051,16011,3000,20781,116503
2,3,Brent,58017,14381,1749,3250,28589,105986
3,4,Tower Hamlets,6787,2442,81377,8109,5786,104501
4,5,Harrow,63051,7797,1378,2629,26953,101808


## Add Coordinates

In [27]:
borough_df = pd.DataFrame( data = {

'London Borough' : ['Newham', 'Redbridge', 'Brent', 'Tower Hamlets', 'Harrow',
                'Ealing', 'Hounslow', 'Hillingdon', 'Barnet', 'Croydon', 'Waltham Forest',
                'Merton', 'Camden', 'Enfield', 'Wandsworth', 'Westminster', 'Greenwich',
                'Barking and Dagenham', 'Southwark', 'Kingston Upon Thames'],
'Latitude' : ['51.5255','51.5901','51.5673','51.5203','51.5806','51.5131','51.4609','51.5352','51.6251','51.3762','51.5886',
             '51.4098','51.5517','51.6636','51.4571','51.4975','51.4934','51.5465','51.4834','51.4123'],
'Longitude' : ['0.0352','0.0819','-0.2711','-0.0293','-0.3420','-0.3256','-0.3731','-0.4481','-0.1529','-0.0982','-0.0118',
              '-0.2108','-0.1588','-0.1043','-0.1818','-0.1357','0.0098','0.1293','-0.0821','-0.3007']})

borough_df.head()

,London Borough,Latitude,Longitude
0,Newham,51.5255,0.0352
1,Redbridge,51.5901,0.0819
2,Brent,51.5673,-0.2711
3,Tower Hamlets,51.5203,-0.0293
4,Harrow,51.5806,-0.3420


In [4]:
borough_df['Latitude'] = borough_df['Latitude'].astype('float', errors = 'ignore')
borough_df['Longitude'] = borough_df['Longitude'].astype('float', errors = 'ignore')
borough_df.dtypes

Borough       object
Latitude     float64
Longitude    float64
dtype: object

In [5]:
address = 'London, UK'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of London are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of London are 51.5073219, -0.1276474.


In [6]:
#Create map
London_map = folium.Map(location=[latitude, longitude], zoom_start=10)

#Add markers to map
for lat, lng, borough in zip(borough_df['Latitude'], borough_df['Longitude'], borough_df['Borough']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(London_map)  
    
London_map

## Explore Data with Foursquare

In [7]:
#Define Foursquare credentials
CLIENT_ID = '0IITJSNBXNENNQMBOIQCU3KACL1J2UYHUVAFHXG2TWYNR4FQ'
CLIENT_SECRET = 'JGLKTYNVRNGY5OP0Q22AYJJKU3HT5BKYOXLKHHDSUBH3CWMZ'
VERSION = '20180605' 

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 0IITJSNBXNENNQMBOIQCU3KACL1J2UYHUVAFHXG2TWYNR4FQ
CLIENT_SECRET:JGLKTYNVRNGY5OP0Q22AYJJKU3HT5BKYOXLKHHDSUBH3CWMZ


In [8]:
#Get first borough
borough_df.loc[0, 'Borough']

'Newham'

In [9]:
#First borough coordinates
borough_latitude = borough_df.loc[0, 'Latitude'] # borough latitude value
borough_longitude = borough_df.loc[0, 'Longitude'] # borough longitude value

borough_name = borough_df.loc[0, 'Borough'] # borough name

print('Latitude and longitude values of {} are {}, {}.'.format(borough_name, 
                                                               borough_latitude, 
                                                               borough_longitude))

Latitude and longitude values of Newham are 51.5255, 0.0352.


In [10]:
#Top 100 venues nearby
LIMIT= 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    borough_latitude, 
    borough_longitude, 
    radius, 
    LIMIT)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e12297947e0d6001b5a637c'},
 'response': {'headerLocation': 'Newham',
  'headerFullLocation': 'Newham, London',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 7,
  'suggestedBounds': {'ne': {'lat': 51.5300000045, 'lng': 0.04241929604043637},
   'sw': {'lat': 51.5209999955, 'lng': 0.027980703959563636}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e7b2628887783148edecd7d',
       'name': 'Delicious Café',
       'location': {'address': '566 Barking Road',
        'lat': 51.526416732217484,
        'lng': 0.030133265936175827,
        'labeledLatLngs': [{'label': 'display',
          'lat': 51.526416732217484,
          'lng': 0.030133265936175827}],
        'distance': 365,
        'postalCode': 'E13 9JU',
      

In [11]:
#Extract category of venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
#Clean data
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Delicious Café,Café,51.526417,0.030133
1,Tesco Express,Grocery Store,51.527187,0.035118
2,Deep Blue Sea Fish & Chips,Fish & Chips Shop,51.525097,0.039410
3,West Ham Food Centre,Convenience Store,51.527451,0.031644
4,Ginny's Pie and Mash,Café,51.525705,0.029532


In [12]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

7 venues were returned by Foursquare.


### Explore boroughs in London

In [13]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough', 
                  'Borough Latitude', 
                  'Borough Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [14]:
London_venues = getNearbyVenues(names= borough_df['Borough'],
                                   latitudes= borough_df['Latitude'],
                                   longitudes= borough_df['Longitude']
                                  )

Newham
Redbridge
Brent
Tower Hamlets
Harrow
Ealing
Hounslow
Hillingdon
Barnet
Croydon
Waltham Forest
Merton
Camden
Enfield
Wandsworth
Westminster
Greenwich
Barking and Dagenham
Southwark
Kingston Upon Thames


In [15]:
print(London_venues.shape)
London_venues.head()

(409, 7)


,Borough,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Newham,51.5255,0.0352,Delicious Café,51.526417,0.030133,Café
1,Newham,51.5255,0.0352,Tesco Express,51.527187,0.035118,Grocery Store
2,Newham,51.5255,0.0352,Deep Blue Sea Fish & Chips,51.525097,0.039410,Fish & Chips Shop
3,Newham,51.5255,0.0352,West Ham Food Centre,51.527451,0.031644,Convenience Store
4,Newham,51.5255,0.0352,Ginny's Pie and Mash,51.525705,0.029532,Café


In [16]:
London_venues.groupby('Borough').count()

,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Borough,,,,,,
Barking and Dagenham,4,4,4,4,4,4
Barnet,3,3,3,3,3,3
Brent,7,7,7,7,7,7
Camden,27,27,27,27,27,27
Croydon,58,58,58,58,58,58
Ealing,25,25,25,25,25,25
Enfield,5,5,5,5,5,5
Greenwich,22,22,22,22,22,22
Harrow,31,31,31,31,31,31


### Analyse each Borough

In [17]:
#One hot encoding
london_onehot = pd.get_dummies(London_venues[['Venue Category']], prefix="", prefix_sep="")

#Add borough column back to dataframe
london_onehot['Borough'] = London_venues['Borough'] 

#Move borough column to the first column
fixed_columns = [london_onehot.columns[-1]] + list(london_onehot.columns[:-1])
london_onehot = london_onehot[fixed_columns]

print(london_onehot.shape)
london_onehot.head()

(409, 123)


,Borough,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,Bagel Shop,Bakery,Bar,Beer Bar,Bookstore,Breakfast Spot,Brewery,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Café,Canal Lock,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Doner Restaurant,Donut Shop,Electronics Store,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Furniture / Home Store,Gastropub,Gift Shop,Go Kart Track,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health Food Store,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Juice Bar,Kebab Restaurant,Kitchen Supply Store,Korean Restaurant,Lake,Lebanese Restaurant,Locksmith,Malay Restaurant,Market,Martial Arts Dojo,Middle Eastern Restaurant,Modern European Restaurant,Movie Theater,Museum,Nightclub,Park,Pedestrian Plaza,Persian Restaurant,Pharmacy,Pier,Pizza Place,Platform,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Public Art,Record Shop,Recreation Center,Restaurant,Rock Climbing Spot,Rock Club,Salad Place,Sandwich Place,Shopping Mall,Skate Park,Smoothie Shop,Spa,Sporting Goods Shop,Stationery Store,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tapas Restaurant,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Train Station,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Wine Shop,Women's Store
0,Newham,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Newham,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Newham,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Newham,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Newham,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [18]:
#Group rows by Borough
london_grouped = london_onehot.groupby('Borough').mean().reset_index()
print(london_grouped.shape)
london_grouped

(20, 123)


,Borough,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,Bagel Shop,Bakery,Bar,Beer Bar,Bookstore,Breakfast Spot,Brewery,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Café,Canal Lock,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Doner Restaurant,Donut Shop,Electronics Store,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Furniture / Home Store,Gastropub,Gift Shop,Go Kart Track,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health Food Store,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Juice Bar,Kebab Restaurant,Kitchen Supply Store,Korean Restaurant,Lake,Lebanese Restaurant,Locksmith,Malay Restaurant,Market,Martial Arts Dojo,Middle Eastern Restaurant,Modern European Restaurant,Movie Theater,Museum,Nightclub,Park,Pedestrian Plaza,Persian Restaurant,Pharmacy,Pier,Pizza Place,Platform,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Public Art,Record Shop,Recreation Center,Restaurant,Rock Climbing Spot,Rock Club,Salad Place,Sandwich Place,Shopping Mall,Skate Park,Smoothie Shop,Spa,Sporting Goods Shop,Stationery Store,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tapas Restaurant,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Train Station,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Wine Shop,Women's Store
0,Barking and Dagenham,0.000000,0.0000,0.0000,0.000000,0.0000,0.000000,0.0,0.00,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.5,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0000,0.0000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000,0.000000,0.0000,0.0000,0.000000,0.250000,0.0,0.000000,0.000000,0.0000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.0000,0.0,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0000,0.0,0.000000,0.000000,0.0000,0.000000,0.000000,0.0000,0.000000,0.0000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000
1,Barnet,0.000000,0.0000,0.0000,0.000000,0.0000,0.000000,0.0,0.00,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.666667,0.0,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0000,0.0000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000,0.000000,0.0000,0.0000,0.000000,0.000000,0.0,0.000000,0.000000,0.0000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0000,0.0,0.000000,0.000000,0.0000,0.000000,0.000000,0.0000,0.000000,0.0000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000
2,Brent,0.000000,0.0000,0.0000,0.000000,0.0000,0.000000,0.0,0.00,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.142857,0.142857,0.142857,0.0,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0000,0.000000,0.000000,0.000000,0.0000

### Top 5 venues per borough

In [19]:
num_top_venues = 5

for hood in london_grouped['Borough']:
    print("----"+hood+"----")
    temp = london_grouped[london_grouped['Borough'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Barking and Dagenham----
                        venue  freq
0  Construction & Landscaping  0.50
1                        Lake  0.25
2                        Park  0.25
3         American Restaurant  0.00
4                      Museum  0.00


----Barnet----
                   venue  freq
0                   Café  0.67
1               Bus Stop  0.33
2    American Restaurant  0.00
3             Public Art  0.00
4  Portuguese Restaurant  0.00


----Brent----
                    venue  freq
0  Furniture / Home Store  0.14
1                Bus Stop  0.14
2                    Café  0.14
3             Supermarket  0.14
4              Food Truck  0.14


----Camden----
         venue  freq
0         Café  0.07
1  Pizza Place  0.07
2     Pharmacy  0.07
3       Bakery  0.07
4    Gastropub  0.07


----Croydon----
            venue  freq
0     Coffee Shop  0.17
1        Platform  0.09
2  Clothing Store  0.09
3           Hotel  0.05
4             Pub  0.05


----Ealing----
                  venu

In [20]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [21]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Borough']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
borough_venues_sorted = pd.DataFrame(columns=columns)
borough_venues_sorted['Borough'] = london_grouped['Borough']

for ind in np.arange(london_grouped.shape[0]):
    borough_venues_sorted.iloc[ind, 1:] = return_most_common_venues(london_grouped.iloc[ind, :], num_top_venues)

borough_venues_sorted

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Barking and Dagenham,Construction & Landscaping,Lake,Park,Women's Store,Food & Drink Shop,Dessert Shop,Diner,Doner Restaurant,Donut Shop,Electronics Store
1,Barnet,Café,Bus Stop,Women's Store,Food & Drink Shop,Dessert Shop,Diner,Doner Restaurant,Donut Shop,Electronics Store,Falafel Restaurant
2,Brent,Middle Eastern Restaurant,Bus Station,Bus Stop,Café,Food Truck,Supermarket,Furniture / Home Store,Fish & Chips Shop,Field,Women's Store
3,Camden,Gastropub,Pharmacy,Bakery,Pizza Place,Coffee Shop,Café,American Restaurant,Burger Joint,Gym / Fitness Center,Wine Shop
4,Croydon,Coffee Shop,Clothing Store,Platform,Hotel,Bookstore,Pub,Portuguese Restaurant,Sushi Restaurant,Dessert Shop,Nightclub
5,Ealing,Hotel,Grocery Store,Fast Food Restaurant,Supermarket,Coffee Shop,Indian Restaurant,Bus Stop,Wine Shop,Furniture / Home Store,Fish Market
6,Enfield,Restaurant,Pub,Coffee Shop,Auto Workshop,Tennis Court,Donut Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Electronics Store
7,Greenwich,Park,Chinese Restaurant,Nightclub,Hotel,Café,Bus Stop,Spa,Pub,Brewery,Portuguese Restaurant
8,Harrow,Coffee Shop,Clothing Store,Gym,Pizza Place,Women's Store,Pub,Pharmacy,Movie Theater,Middle Eastern Restaurant,Italian Restaurant
9,Hillingdon,Burger Joint,Chinese Restaurant,Martial Arts Dojo,Construction & Landscaping,Breakfast Spot,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market


## Cluster Boroughs

In [83]:
# set number of clusters
kclusters = 5

london_grouped_clustering = london_grouped.drop('Borough', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(london_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 2, 3, 3, 3, 3, 3, 3, 3, 4], dtype=int32)

In [84]:
#New DF to cluster populations and venues
london_merged = London_df.merge(borough_venues_sorted, how='inner', left_on='London Borough', right_on='Borough')
london_merged['Cluster Labels'] = kmeans.labels_
london_merged.head()

,Rank,London Borough,Indian Population,Pakistani Population,Bangladeshi Population,Chinese Population,Other Asian Population,Total Asian Population,Cluster Labels,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,Newham,42484,30307,37262,3930,19912,133895,1,Newham,Grocery Store,Café,Convenience Store,Park,Fish & Chips Shop,Fish Market,Department Store,Dessert Shop,Diner,Doner Restaurant
1,2,Redbridge,45660,31051,16011,3000,20781,116503,2,Redbridge,Indian Restaurant,Supermarket,Pharmacy,Pub,Coffee Shop,Park,Sandwich Place,Café,Middle Eastern Restaurant,Fast Food Restaurant
2,3,Brent,58017,14381,1749,3250,28589,105986,3,Brent,Middle Eastern Restaurant,Bus Station,Bus Stop,Café,Food Truck,Supermarket,Furniture / Home Store,Fish & Chips Shop,Field,Women's Store
3,4,Tower Hamlets,6787,2442,81377,8109,5786,104501,3,Tower Hamlets,Canal Lock,Coffee Shop,Go Kart Track,Thrift / Vintage Store,Persian Restaurant,Steakhouse,Skate Park,Recreation Center,Lebanese Restaurant,Museum
4,5,Harrow,63051,7797,1378,2629,26953,101808,3,Harrow,Coffee Shop,Clothing Store,Gym,Pizza Place,Women's Store,Pub,Pharmacy,Movie Theater,Middle Eastern Restaurant,Italian Restaurant


In [85]:
#Add coordinates
london_merged[['Latitude', 'Longitude']] = borough_df[['Latitude', 'Longitude']]
london_merged['Latitude'] = london_merged['Latitude'].astype(float)
london_merged['Longitude'] = london_merged['Longitude'].astype(float)
london_merged.head()

,Rank,London Borough,Indian Population,Pakistani Population,Bangladeshi Population,Chinese Population,Other Asian Population,Total Asian Population,Cluster Labels,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Latitude,Longitude
0,1,Newham,42484,30307,37262,3930,19912,133895,1,Newham,Grocery Store,Café,Convenience Store,Park,Fish & Chips Shop,Fish Market,Department Store,Dessert Shop,Diner,Doner Restaurant,51.5255,0.0352
1,2,Redbridge,45660,31051,16011,3000,20781,116503,2,Redbridge,Indian Restaurant,Supermarket,Pharmacy,Pub,Coffee Shop,Park,Sandwich Place,Café,Middle Eastern Restaurant,Fast Food Restaurant,51.5901,0.0819
2,3,Brent,58017,14381,1749,3250,28589,105986,3,Brent,Middle Eastern Restaurant,Bus Station,Bus Stop,Café,Food Truck,Supermarket,Furniture / Home Store,Fish & Chips Shop,Field,Women's Store,51.5673,-0.2711
3,4,Tower Hamlets,6787,2442,81377,8109,5786,104501,3,Tower Hamlets,Canal Lock,Coffee Shop,Go Kart Track,Thrift / Vintage Store,Persian Restaurant,Steakhouse,Skate Park,Recreation Center,Lebanese Restaurant,Museum,51.5203,-0.0293
4,5,Harrow,63051,7797,1378,2629,26953,101808,3,Harrow,Coffee Shop,Clothing Store,Gym,Pizza Place,Women's Store,Pub,Pharmacy,Movie Theater,Middle Eastern Restaurant,Italian Restaurant,51.5806,-0.3420


In [86]:
#Visualise clusters
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)


x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]


markers_colors = []
for lat, lon, poi, cluster in zip(london_merged['Latitude'], london_merged['Longitude'], london_merged['Borough'], london_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

### Cluster 1

In [87]:
london_merged.loc[london_merged['Cluster Labels'] == 0, london_merged.columns[[0] + list(range(1, london_merged.shape[1]))]].sort_values(by = ['Chinese Population'], ascending = False)

,Rank,London Borough,Indian Population,Pakistani Population,Bangladeshi Population,Chinese Population,Other Asian Population,Total Asian Population,Cluster Labels,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Latitude,Longitude
18,19,Southwark,5819,1623,3912,8074,7764,27192,0,Southwark,Park,Building,Pub,Skate Park,Café,Lake,Falafel Restaurant,Fast Food Restaurant,Farmers Market,Women's Store,51.4834,-0.0821
12,13,Camden,6083,1489,12503,6493,8878,35446,0,Camden,Gastropub,Pharmacy,Bakery,Pizza Place,Coffee Shop,Café,American Restaurant,Burger Joint,Gym / Fitness Center,Wine Shop,51.5517,-0.1588
13,14,Enfield,11648,2594,5599,2588,12464,34893,0,Enfield,Restaurant,Pub,Coffee Shop,Auto Workshop,Tennis Court,Donut Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Electronics Store,51.6636,-0.1043
17,18,Barking and Dagenham,7436,8007,7701,1315,5135,29594,0,Barking and Dagenham,Construction & Landscaping,Lake,Park,Women's Store,Food & Drink Shop,Dessert Shop,Diner,Doner Restaurant,Donut Shop,Electronics Store,51.5465,0.1293


### Cluster 2

In [88]:
london_merged.loc[london_merged['Cluster Labels'] == 1, london_merged.columns[[0] + list(range(1, london_merged.shape[1]))]].sort_values(by = ['Chinese Population'], ascending = False)

,Rank,London Borough,Indian Population,Pakistani Population,Bangladeshi Population,Chinese Population,Other Asian Population,Total Asian Population,Cluster Labels,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Latitude,Longitude
0,1,Newham,42484,30307,37262,3930,19912,133895,1,Newham,Grocery Store,Café,Convenience Store,Park,Fish & Chips Shop,Fish Market,Department Store,Dessert Shop,Diner,Doner Restaurant,51.5255,0.0352


### Cluster 3

In [89]:
london_merged.loc[london_merged['Cluster Labels'] == 2, london_merged.columns[[0] + list(range(1, london_merged.shape[1]))]].sort_values(by = ['Chinese Population'], ascending = False)

,Rank,London Borough,Indian Population,Pakistani Population,Bangladeshi Population,Chinese Population,Other Asian Population,Total Asian Population,Cluster Labels,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Latitude,Longitude
1,2,Redbridge,45660,31051,16011,3000,20781,116503,2,Redbridge,Indian Restaurant,Supermarket,Pharmacy,Pub,Coffee Shop,Park,Sandwich Place,Café,Middle Eastern Restaurant,Fast Food Restaurant,51.5901,0.0819


### Cluster 4

In [90]:
london_merged.loc[london_merged['Cluster Labels'] == 3, london_merged.columns[[0] + list(range(1, london_merged.shape[1]))]].sort_values(by = ['Chinese Population'], ascending = False)

,Rank,London Borough,Indian Population,Pakistani Population,Bangladeshi Population,Chinese Population,Other Asian Population,Total Asian Population,Cluster Labels,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Latitude,Longitude
8,9,Barnet,27920,5344,2215,8259,22180,65918,3,Barnet,Café,Bus Stop,Women's Store,Food & Drink Shop,Dessert Shop,Diner,Doner Restaurant,Donut Shop,Electronics Store,Falafel Restaurant,51.6251,-0.1529
3,4,Tower Hamlets,6787,2442,81377,8109,5786,104501,3,Tower Hamlets,Canal Lock,Coffee Shop,Go Kart Track,Thrift / Vintage Store,Persian Restaurant,Steakhouse,Skate Park,Recreation Center,Lebanese Restaurant,Museum,51.5203,-0.0293
15,16,Westminster,7213,2328,6299,5917,10105,31862,3,Westminster,Coffee Shop,Hotel,Sandwich Place,Sushi Restaurant,Theater,Pub,Café,Juice Bar,Sporting Goods Shop,Indian Restaurant,51.4975,-0.1357
16,17,Greenwich,7836,2594,1645,5061,12758,29894,3,Greenwich,Park,Chinese Restaurant,Nightclub,Hotel,Café,Bus Stop,Spa,Pub,Brewery,Portuguese Restaurant,51.4934,0.0098
5,6,Ealing,48240,14711,1786,4132,31570,100439,3,Ealing,Hotel,Grocery Store,Fast Food Restaurant,Supermarket,Coffee Shop,Indian Restaurant,Bus Stop,Wine Shop,Furniture / Home Store,Fish Market,51.5131,-0.3256
14,15,Wandsworth,8642,9718,1493,3715,9770,33338,3,Wandsworth,Café,Pub,Grocery Store,Coffee Shop,Indian Restaurant,Park,Pizza Place,Hotel,Bagel Shop,Food & Drink Shop,51.4571,-0.1818
2,3,Brent,58017,14381,1749,3250,28589,105986,3,Brent,Middle Eastern Restaurant,Bus Station,Bus Stop,Café,Food Truck,Supermarket,Furniture / Home Store,Fish & Chips Shop,Field,Women's Store,51.5673,-0.2711
7,8,Hillingdon,36795,9200,2639,2889,17730,69253,3,Hillingdon,Burger Joint,Chinese Restaurant,Martial Arts Dojo,Construction & Landscaping,Breakfast Spot,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,51.5352,-0.4481
19,20,Kingston Upon Thames,6325,3009,892,2883,13043,26152,3,Kingston Upon Thames,Coffee Shop,Café,Clothing Store,Sushi Restaurant,Department Store,Bookstore,Bakery,Japanese Restaurant,Italian Restaurant,Thai Restaurant,51.4123,-0.3007
4,5,Harrow,63051,7797,1378,2629,26953,101808,3,Harrow,Coffee Shop,Clothing Store,Gym,Pizza Place,Women's Store,Pub,Pharmacy,Movie Theater,Middle Eastern Restaurant,Italian Restaurant,51.5806,-0.3420


### Cluster 5

In [91]:
london_merged.loc[london_merged['Cluster Labels'] == 4, london_merged.columns[[0] + list(range(1, london_merged.shape[1]))]].sort_values(by = ['Chinese Population'], ascending = False)

,Rank,London Borough,Indian Population,Pakistani Population,Bangladeshi Population,Chinese Population,Other Asian Population,Total Asian Population,Cluster Labels,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Latitude,Longitude
9,10,Croydon,24660,10865,2570,3925,17607,59627,4,Croydon,Coffee Shop,Clothing Store,Platform,Hotel,Bookstore,Pub,Portuguese Restaurant,Sushi Restaurant,Dessert Shop,Nightclub,51.3762,-0.0982
